In [ ]:
#Importing Needed Libaries
import nltk
import numpy as np


In [ ]:
#importing data for nltk
nltk.download('punkt')

In [ ]:
# Input String
inputStr='''add 3, 5 .
print result.
'''
print("\nInput String :-\n",inputStr)
# inputStr

In [ ]:
#making Tokens of input 
tokenizeInput=np.array(nltk.word_tokenize(inputStr))
print("\nTokenzied Input :- \n",tokenizeInput,"\n")

In [ ]:
# CustomTree and MakeTape

class CustomTree():
    def __init__(self):
        self.left=None
        self.right=None
        self.data=None
    
    def insertNode(self,value):
        if self.data is None:
            self.data=value
        elif self.right is None:
            # print("right is none ",value)
            self.right=CustomTree()
            self.right.insertNode(value=value)
        else:
            if "$" in self.right.data:
                # print("right is not none and $ ",value)
                temp=self.right
                self.right=CustomTree()
                self.right.insertNode(value=value)
                self.right.left=temp
            else:
                # print("right is not none ",value)
                self.right.insertNode(value)
    def PrintTree(self,nodePostion="root",height=0):
        print(f'{self.data} : {nodePostion}, Height : {height}'),
        if self.left:
            self.left.PrintTree(nodePostion="left",height=height+1)
        if self.right:
            self.right.PrintTree(nodePostion="right",height=height+1)

def makeTape(node:CustomTree):
    tape=[]
    while node.data is not None:
        tape.append(node.data)
        if node.left is not None:
            tape.append(node.left.data)
        if node.right is not None:
            node=node.right
        else:
            break
    return tape

In [ ]:
#Word as function For Arthimetic Env
class ArithameticEnv:
    def __init__(self):
        pass
    @staticmethod
    def addFun(*args):
        result =" ".join(["result =", *[f'{x if "$" not in x else x[1:]} +' for x in args]])
        result=result[:-1]
        return result
    @staticmethod
    def andFun(*args):
        return None
    @staticmethod
    def commaFun(*args):
        ArithameticEnv.andFun(*args)
    @staticmethod
    def printfun(*args):
        result =" ".join(["print(", *[f'{x if "$" not in x else x[1:]} ' for x in args],")"])
        return result
    env_Variables=["result"]
    env_Words_and_WordAsFunction={"add":addFun.__func__,"and":andFun.__func__,",":commaFun.__func__,"print":printfun.__func__}
# ArithameticEnv.addFun("v1","v2")
#ArithameticEnv.addFun("$v1","v1")
# ArithameticEnv.env_Words_and_WordAsFunction['add']("$v1","v1")

In [ ]:
# Making of Tape from Sentence
global_Tape=[]
itreate_each_word=iter(tokenizeInput)
catch_variable_value=False
end_Of_a_Sentence=True
node=None
try:
    while True:
        current_Word=next(itreate_each_word)
        if "$"== current_Word and not catch_variable_value:
            catch_variable_value=True
        elif "."==current_Word and not end_Of_a_Sentence:
            end_Of_a_Sentence=True
            print("\nCustom Tree Stracture of the current Sentence :- \n")
            node.PrintTree()
            tape=makeTape(node=node)
            global_Tape.append(tape)
            print("\nEnd of a sentence","\nTape :- \n",tape,"\n")
        else:
            if end_Of_a_Sentence:
                print("\nStarting of a sentence\n")
                end_Of_a_Sentence=False
                node=CustomTree()
            elif catch_variable_value:
                catch_variable_value=False
                current_Word="$"+current_Word
            elif str(current_Word).isnumeric():
                current_Word="$"+current_Word
            elif current_Word in ArithameticEnv.env_Variables:
                current_Word="$"+current_Word
            node.insertNode(current_Word)
            # print(current_Word)
except StopIteration:
    print("\nEnd of All the lines\n")
    print("\nGlobal Tape :-\n",global_Tape)
    pass


In [ ]:
#The machice to Transfer the natural languae to Python code
pythonCode=[]
varibleBuffer=[]
for sentenceTape in global_Tape:
    varibleBuffer=[]
    for word in reversed(sentenceTape):
        if "$" in word :
            varibleBuffer.append(word)
        else:
            result=ArithameticEnv.env_Words_and_WordAsFunction[word](*varibleBuffer)
            if result is not None:
               pythonCode.append(result)
# pythonCode






In [ ]:
print("\npython code Exected Output\n")
exec("\n".join(pythonCode))